<a href="https://colab.research.google.com/github/RepnaiaE/DataScience_Emotion-recognition-methods/blob/main/Lab6_Neural_network_of_direct_distribution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Нейронные сети. Нейронная сеть прямого распространения./Neural networks. Neural network of direct distribution.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Загрузка данных/Loading data

Загружаем данные из ячеек в массивы признаков для каждого набора из предыдущей работы

In [ ]:
import pandas as pd

negative = pd.read_csv('/content/drive/MyDrive/Распознавание эмоций/negative.csv', sep=';',
                       names=['id', 'tdate', 'tname', 'ttext', 'ttype', 'trep', 'trtv',
                              'tfav', 'tstcount', 'tfol', 'tfrien', 'listcount'])
negative.head()

,id,tdate,tname,ttext,ttype,trep,trtv,tfav,tstcount,tfol,tfrien,listcount
0,408906762813579264,1386325944,dugarchikbellko,на работе был полный пиддес :| и так каждое за...,-1,0,0,0,8064,111,94,2
1,408906818262687744,1386325957,nugemycejela,"Коллеги сидят рубятся в Urban terror, а я из-з...",-1,0,0,0,26,42,39,0
2,408906858515398656,1386325966,4post21,@elina_4post как говорят обещаного три года жд...,-1,0,0,0,718,49,249,0
3,408906914437685248,1386325980,Poliwake,"Желаю хорошего полёта и удачной посадки,я буду...",-1,0,0,0,10628,207,200,0
4,408906914723295232,1386325980,capyvixowe,"Обновил за каким-то лешим surf, теперь не рабо...",-1,0,0,0,35,17,34,0


In [ ]:
positive = pd.read_csv('/content/drive/MyDrive/Распознавание эмоций/positive.csv', sep=';',
                       names=['id', 'tdate', 'tname', 'ttext', 'ttype', 'trep', 'trtv',
                              'tfav', 'tstcount', 'tfol', 'tfrien', 'listcount'])
positive.head()

,id,tdate,tname,ttext,ttype,trep,trtv,tfav,tstcount,tfol,tfrien,listcount
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1


In [ ]:
DATA_LEN = 15000

data = pd.concat([positive.head(DATA_LEN), negative.head(DATA_LEN)])
data.loc[(data.ttype < 0), ('ttype')] = 0
data

,id,tdate,tname,ttext,ttype,trep,trtv,tfav,tstcount,tfol,tfrien,listcount
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1
...,...,...,...,...,...,...,...,...,...,...,...,...
14995,411052214937874432,1386837459,nataliyatm,"@kosaya_lisa я бы была рада морозу больше, чем...",0,0,0,0,7886,62,100,0
14996,411052217857474560,1386837460,vynujapyzm,Мне противно смотреть на логи нашего провайдер...,0,0,0,0,281,264,256,0
14997,411052251843936256,1386837468,gubapikahacu,"RT @fupefodicita: ""Большинство россиян по-преж...",0,0,1,0,853,273,257,0
14998,411052507985879040,1386837529,cyberr98,@Yulia_Demchuk @Leno_ok1998 давайте в школу мн...,0,0,0,0,678,54,38,0


In [ ]:
X=data['ttext'].values.tolist()
y=data['ttype'].values.tolist()

# Обучение языковой модели/Language model training

In [ ]:
from string import punctuation
import nltk
nltk.download('punkt')

import nltk.corpus
import nltk.tokenize

punctuations = list(punctuation)
punkt = ['``','...',"''",'«','»','…','”','”','“','-','–','..']
punctuations.extend(punkt)

def tokenize(text):
    text_token = nltk.word_tokenize(text)
    text_word=[x.lower() for x in text_token if x not in punctuations and x.isalpha]
    return text_word

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
X_token = [tokenize(t) for t in X]

In [ ]:
VECTOR_SIZE = 300

In [ ]:
!pip install --upgrade gensim

In [ ]:
from gensim.models import Word2Vec

w2v_model = Word2Vec(sentences=X_token, vector_size=VECTOR_SIZE, window=5, min_count=1, workers=4)

In [ ]:
w2v_model.build_vocab(X_token)

In [ ]:
w2v_model.train(X_token, total_examples=w2v_model.corpus_count,
                epochs=10, report_delay=1)
w2v_model.init_sims(replace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
import numpy as np
import gensim

def word_averaging(w2v_model, words):
    mean = []

    for word in words:
        m = w2v_model.wv[word]
        mean.append(m)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

In [ ]:
def word_averaging_list(model, text_list):
    return np.vstack([word_averaging(model, comment_text) for comment_text in text_list])

In [ ]:
X = word_averaging_list(w2v_model, X_token)
X.shape

(30000, 300)

In [ ]:
from sklearn.model_selection import train_test_split
import torch
import torch.utils.data as data_utils

def getDataLoaders(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                        test_size=0.2)
    inputs_train = torch.tensor(X_train)
    targets_train = torch.IntTensor(y_train)

    inputs_test = torch.tensor(X_test)
    targets_test = torch.IntTensor(y_test)

    train = data_utils.TensorDataset(inputs_train, targets_train)
    test = data_utils.TensorDataset(inputs_test, targets_test)

    trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=16)
    testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=16)

    return trainset, testset, y_test

In [ ]:
train_set, test_set, test_y = getDataLoaders(X, y)

# Модель/Model

In [ ]:
NUM_CLASSES = 2

In [ ]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.metrics import classification_report

class Net(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, activation=F.log_softmax):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        self.act = activation

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return self.act(x, dim=1)

In [ ]:
w2vNet = Net(VECTOR_SIZE, 128, NUM_CLASSES)
print(w2vNet)

Net(
  (fc1): Linear(in_features=300, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=2, bias=True)
)


# Обучение/Training

In [ ]:
import torch.optim as optim
import torch.nn.functional as F
from sklearn.metrics import classification_report

from tqdm import tqdm

def train_test(net, trainset, testset, y_test,
          epochs=3,
          loss_function=nn.NLLLoss(),
          optimizer=None):

    if optimizer is None:
        optimizer = optim.Adam(net.parameters(), lr=0.0002)

    # train
    for epoch in range(epochs):
        for data in tqdm(trainset):
            X, y = data

            net.zero_grad()
            output = net(X.float())
            loss = F.nll_loss(output, y.long())
            loss.backward()
            optimizer.step()

        print(f"train loss {loss} for epoch {epoch}")

    # test
    ams = []
    with torch.no_grad():
        for data in testset:
            X, y = data

            output = net(X.float())
            for idx, i in enumerate(output):
                ams.append(torch.argmax(i).item())

#     print(ams)
#     print("-------------------")
#     print(y_test)
    print(classification_report(y_test, ams))

In [ ]:
train_test(w2vNet, train_set, test_set, test_y, epochs=10)

100%|██████████| 16000/16000 [00:32<00:00, 498.36it/s]


train loss 0.674863338470459 for epoch 0


100%|██████████| 16000/16000 [00:29<00:00, 544.91it/s]


train loss 0.6840765476226807 for epoch 1


100%|██████████| 16000/16000 [00:37<00:00, 425.37it/s]


...

100%|██████████| 16000/16000 [00:37<00:00, 430.71it/s]


train loss 0.717125654220581 for epoch 8


100%|██████████| 16000/16000 [00:35<00:00, 445.20it/s]


train loss 0.6907408833503723 for epoch 9
              precision    recall  f1-score   support

           0       0.50      1.00      0.67      2010
           1       0.00      0.00      0.00      1990

    accuracy                           0.50      4000
   macro avg       0.25      0.50      0.33      4000
weighted avg       0.25      0.50      0.34      4000



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Предобученная языковая модель/Pretrained language model

Использовалась предобученая модель word2vec для русского языка

In [ ]:
model_url = 'http://vectors.nlpl.eu/repository/20/180.zip'

  Using cached gensim-4.2.0-cp39-cp39-win_amd64.whl (23.9 MB)
  Using cached Cython-0.29.28-py2.py3-none-any.whl (983 kB)
  Using cached smart_open-6.0.0-py3-none-any.whl (58 kB)
  Attempting uninstall: Cython
    Found existing installation: Cython 0.29.24
    Uninstalling Cython-0.29.24:
      Successfully uninstalled Cython-0.29.24


In [ ]:
import gensim

model = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/MyDrive/Распознавание эмоций/model.bin', binary=True)

In [ ]:
import pickle

data = open('/content/drive/MyDrive/Распознавание эмоций/sentiment_POS', 'rb')
loaded_data = pickle.load(data)
loaded_data

,ttext,ttype,text_pos,X
0,на работе был полный пиддес :| и так каждое за...,0,на_ADP работа_NOUN быть_AUX полный_ADJ пиддо_N...,"[на_ADP, работа_NOUN, быть_AUX, полный_ADJ, пи..."
1,"Коллеги сидят рубятся в Urban terror, а я из-з...",0,коллега_NOUN сидеть_VERB рубиться_VERB в_ADP u...,"[коллега_NOUN, сидеть_VERB, рубиться_VERB, в_A..."
2,@elina_4post как говорят обещаного три года жд...,0,@elina4post_X как_SCONJ говорить_VERB обещаный...,"[@elina4post_X, как_SCONJ, говорить_VERB, обещ..."
3,"Желаю хорошего полёта и удачной посадки,я буду...",0,желать_VERB хороший_ADJ полет_NOUN и_CCONJ уда...,"[желать_VERB, хороший_ADJ, полет_NOUN, и_CCONJ..."
4,"Обновил за каким-то лешим surf, теперь не рабо...",0,обновить_VERB за_ADP какой-то_ADJ лешимый_ADJ ...,"[обновить_VERB, за_ADP, какой-то_ADJ, лешимый_..."
...,...,...,...,...
1999,уаааа)))))) сейчас около меня прошел паренек с...,1,уааа_NOUN сейчас_ADV около_ADP я_PRON проходит...,"[уааа_NOUN, сейчас_ADV, около_ADP, я_PRON, про..."
2000,За монгол коментийн 2 оо vзэж жаахан хэрэгт ду...,1,за_ADP монгол_NOUN коментийн_NOUN x_NUM оо_ADV...,"[за_ADP, монгол_NOUN, коментийн_NOUN, x_NUM, о..."
2001,"""@AdelKhalilova: @Nastasseya @AnuTochkaSavina ...",1,@adelkhalilova_PROPN @nastasseya_PROPN @anutoc...,"[@adelkhalilova_PROPN, @nastasseya_PROPN, @anu..."
2002,"Дим, ты помогаешь мне, я тебе, все взаимно, вс...",1,дима_PROPN ты_PRON помогать_VERB я_PRON я_PRON...,"[дима_PROPN, ты_PRON, помогать_VERB, я_PRON, я..."


In [ ]:
import pandas as pd
import numpy as np

def word_averaging(model, words):
    all_words, mean = set(), []

    for word in words:
        if word in model.key_to_index.keys():
            mean.append(model[model.key_to_index[word]])
            all_words.add(model.key_to_index[word])
        else:
            mean.append(np.zeros(w2v_model.vector_size))

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

In [ ]:
def word_averaging_list(model, text_list):
    return np.vstack([word_averaging(model, comment_text) for comment_text in text_list ])

In [ ]:
y = loaded_data['ttype'].values.tolist()

X=loaded_data['X']
X = word_averaging_list(model, X)
X.shape

(2004, 300)

In [ ]:
train_set, test_set, test_y = getDataLoaders(X, y)

In [ ]:
w2vNet = Net(VECTOR_SIZE, 128, NUM_CLASSES)
print(w2vNet)

Net(
  (fc1): Linear(in_features=300, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=2, bias=True)
)


In [ ]:
train_test(w2vNet, train_set, test_set, test_y, epochs=7)

100%|█████████████████████████████████████████████████████████████████████████████| 1603/1603 [00:01<00:00, 813.28it/s]


train loss 0.8613996505737305 for epoch 0


100%|█████████████████████████████████████████████████████████████████████████████| 1603/1603 [00:01<00:00, 817.75it/s]


train loss 0.4739820659160614 for epoch 1


100%|█████████████████████████████████████████████████████████████████████████████| 1603/1603 [00:01<00:00, 864.56it/s]


...

train loss 0.17162296175956726 for epoch 4


100%|█████████████████████████████████████████████████████████████████████████████| 1603/1603 [00:02<00:00, 771.10it/s]


train loss 0.037507064640522 for epoch 5


100%|█████████████████████████████████████████████████████████████████████████████| 1603/1603 [00:01<00:00, 923.21it/s]

train loss 0.4917450249195099 for epoch 6
              precision    recall  f1-score   support

           0       0.61      0.65      0.63       198
           1       0.64      0.60      0.62       203

    accuracy                           0.63       401
   macro avg       0.63      0.63      0.63       401
weighted avg       0.63      0.63      0.63       401



# Перебор параметров/Parameter enumeration

In [ ]:
import torch.nn as nn

class Net(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim,
                 activation=F.log_softmax,
                 init_func=nn.init.xavier_uniform_,
                 norm_batch=nn.BatchNorm1d):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)

        self.bn1 = nn.BatchNorm1d(hidden_dim)

        self.act = activation

        init_func(self.fc1.weight)
        init_func(self.fc2.weight)
        init_func(self.fc3.weight)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.bn1(x)

        x = F.relu(self.fc2(x))
        x = self.bn1(x)

        x = self.fc3(x)

        if self.act == F.relu:
            return self.act(x)

        return self.act(x, dim=1)

In [ ]:
import torch.optim as optim
import torch.nn.functional as F
from sklearn.metrics import classification_report

from tqdm import tqdm

def train_test(net, trainset, testset, y_test,
          epochs=3,
          loss_function=nn.NLLLoss(),
          optimizer=None):

    if optimizer is None:
        optimizer = optim.Adam(net.parameters(), lr=0.0002)

    # train
    for epoch in tqdm(range(epochs)):
        for data in trainset:
            X, y = data

            net.zero_grad()
            output = net(X.float())
            loss = F.nll_loss(output, y.long())
            loss.backward()
            optimizer.step()

        print(f"train loss {loss} for epoch {epoch}")

    # test
    ams = []
    with torch.no_grad():
        for data in testset:
            X, y = data

            output = net(X.float())
            for idx, i in enumerate(output):
                ams.append(torch.argmax(i).item())

    print(classification_report(y_test, ams))

In [ ]:
def try_all_params(train_set, test_set, test_y, epochs=5):
    act_funcs = {"softmax": F.log_softmax, "relu": F.relu}
    loss_funcs = {"nll": nn.NLLLoss(), "cross entropy": nn.CrossEntropyLoss()}
    init_funcs = {"xavier": nn.init.xavier_uniform_, "zeros": nn.init.zeros_}
    reg_funcs = {"adam": torch.optim.Adam, "sgd": torch.optim.SGD}
    norm_funcs = {"layer norm": nn.LayerNorm, "norm1d": nn.BatchNorm1d}
    lr_values = {"2e-5": 2e-5, "2e-2": 2e-2}

    for n_af, af in act_funcs.items():
        for n_lf, lf in loss_funcs.items():
            for n_if, iff in init_funcs.items():
                for n_nr, nr in norm_funcs.items():
                    for n_rf, rf in reg_funcs.items():
                        for n_lr, lr in lr_values.items():
                            print(f"{n_af}, {n_lf}, {n_if}, {n_nr}, {n_rf}, {n_lr}")

                            w2vNet = Net(VECTOR_SIZE, 32, NUM_CLASSES, activation=af, init_func=iff, norm_batch=nr)
                            train_test(w2vNet, train_set, test_set, test_y, epochs=epochs, loss_function=lf,
                                      optimizer=rf(w2vNet.parameters(), lr=lr))

                            print('-' * 20)

                            torch.cuda.empty_cache()
                            del w2vNet

In [ ]:
# перебор по предобученным векторам

try_all_params(train_set, test_set, test_y, epochs=3)

softmax, nll, xavier, layer norm, adam, 2e-5


 67%|██████▋   | 2/3 [00:00<00:00, 11.45it/s]

train loss 0.8399602770805359 for epoch 0
train loss 0.8480142951011658 for epoch 1


100%|██████████| 3/3 [00:00<00:00, 10.41it/s]

train loss 0.2692756950855255 for epoch 2


...

 67%|██████▋   | 2/3 [00:00<00:00, 13.19it/s]

train loss -1.0827398300170898 for epoch 1


100%|██████████| 3/3 [00:00<00:00, 13.25it/s]

train loss -1.5928090810775757 for epoch 2
              precision    recall  f1-score   support

           0       0.51      1.00      0.68       205
           1       0.00      0.00      0.00       196

    accuracy                           0.51       401
   macro avg       0.26      0.50      0.34       401
weighted avg       0.26      0.51      0.35       401

--------------------



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# перебор по обученной модели

try_all_params(train_set, test_set, test_y, epochs=3)

softmax, nll, xavier, layer norm, adam, 2e-5


 33%|███▎      | 1/3 [00:02<00:04,  2.14s/it]

train loss 0.7712003588676453 for epoch 0


 67%|██████▋   | 2/3 [00:04<00:02,  2.15s/it]

train loss 0.7787870764732361 for epoch 1


...

train loss -4.3115623624369766e+17 for epoch 1


100%|██████████| 3/3 [00:04<00:00,  1.46s/it]

train loss -4.224615757391169e+30 for epoch 2


              precision    recall  f1-score   support

           0       0.50      1.00      0.66      2971
           1       0.00      0.00      0.00      3029

    accuracy                           0.50      6000
   macro avg       0.25      0.50      0.33      6000
weighted avg       0.25      0.50      0.33      6000

--------------------


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Выводы/Conclusion

Было обучено множество моделей на основе Net (описана в пункте Перебор параметров) с перебором параметров для обучения. Обучение проводилось на признаках, выделенных с помощью обученной модели и предобученной.

Many Net-based models were trained (described in Parametric Brute Force) with parameter brute force training. The training was carried out on features extracted using the trained model and the pre-trained one.

## Результаты предобученной модели/Pretrained Model Results

| Features | Params | F1 score |
|----------|--------|----------|
| pretrained w2v | softmax, cross entropy, zeros, norm1d, adam, 2e-2 | 0.70 |
| pretrained w2v | softmax, cross entropy, xavier, layer norm, adam, 2e-2 | 0.67 |
| pretrained w2v | softmax, nll, zeros, norm1d, adam, 2e-2 | 0.66 |

## Результаты обученной модели/Trained Model Results

| Features | Params | F1 score |
|----------|--------|----------|
| w2v | softmax, nll, xavier, layer norm, sgd, 2e-2 | 0.64 |
| w2v | softmax, nll, xavier, norm1d, sgd, 2e-2 | 0.64 |
| w2v | softmax, nll, zeros, norm1d, adam, 2e-2 | 0.64 |